In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import geopandas as gpd
import ast
import os
import json

# 1. Inspect and Load GeoJSON Files
data_dir = "/kaggle/input/treesatai-indices"  # Path to new data
all_features = []
all_labels = []
invalid_samples = []

# Updated list of bands based on inspection
bands = ['ARVI', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'DEM', 'EVI', 'EVI2', 'NDVI', 'NDWI']
months = ['', '_1', '_2', '_3', '_4', '_5', '_6', '_7']
band_columns = [band + month for month in months for band in bands]  # 18 bands × 8 months = 144 channels

# Inspect first file for debugging
first_file = os.path.join(data_dir, os.listdir(data_dir)[0]) if os.listdir(data_dir) else None
if first_file and first_file.endswith(".geojson"):
    gdf = gpd.read_file(first_file)
    print("Inspecting first 2 rows of first GeoJSON file:")
    for idx in range(min(2, len(gdf))):
        print(f"\nRow {idx}:")
        for band in ['ARVI', 'B1', 'B11', 'NDVI', 'B2_1', 'NDVI_7']:  # Sample of new and old bands
            data = gdf[band].iloc[idx]
            print(f"  Band {band}: type={type(data)}, raw_value={str(data)[:100]}...")
            try:
                if isinstance(data, (list, np.ndarray)):
                    array = np.array(data, dtype=np.float32)
                elif isinstance(data, str):
                    try:
                        array = np.array(json.loads(data), dtype=np.float32)
                    except json.JSONDecodeError:
                        array = np.array(ast.literal_eval(data), dtype=np.float32)
                elif isinstance(data, (float, np.float32, np.float64)):
                    array = np.full((5, 5), data, dtype=np.float32)
                else:
                    raise ValueError(f"Unexpected data type for band {band}: {type(data)}")
                if array.shape != (5, 5):
                    raise ValueError(f"Band {band} has shape {array.shape}, expects (5, 5)")
                print(f"  Band {band}: shape={array.shape}, first few values={array.flatten()[:5]}")
            except Exception as e:
                print(f"  Band {band}: Error processing data: {e}")

# Load all GeoJSON files
for file in os.listdir(data_dir):
    if file.endswith(".geojson"):
        gdf = gpd.read_file(os.path.join(data_dir, file))
        for idx, row in gdf.iterrows():
            try:
                patches = []
                for col in band_columns:
                    if col not in gdf.columns:
                        raise ValueError(f"Column {col} not found in {file}")
                    val = row[col]
                    if isinstance(val, (list, np.ndarray)):
                        arr = np.array(val, dtype=np.float32).reshape(5, 5)
                    elif isinstance(val, str):
                        try:
                            arr = np.array(json.loads(val), dtype=np.float32).reshape(5, 5)
                        except json.JSONDecodeError:
                            arr = np.array(ast.literal_eval(val), dtype=np.float32).reshape(5, 5)
                    elif isinstance(val, (float, np.float32, np.float64)):
                        arr = np.full((5, 5), val, dtype=np.float32)
                    else:
                        raise ValueError(f"Unexpected data type for band {col}: {type(val)}")
                    patches.append(arr)

                patch = np.stack(patches, axis=-1)
                if patch.shape != (5, 5, 144):  # Updated to 18 bands × 8 months = 144 channels
                    raise ValueError(f"Unexpected patch shape: {patch.shape}, expected (5, 5, 144)")
                all_features.append(patch)
                all_labels.append(row['l3_species'])
            except Exception as e:
                invalid_samples.append((file, idx, str(e)))
                continue

# Log invalid samples
if invalid_samples:
    print(f"\nSkipped {len(invalid_samples)} invalid samples:")
    for file, idx, error in invalid_samples[:5]:
        print(f"File: {file}, Row: {idx}, Error: {error}")
    if invalid_samples:
        print(f"First invalid sample raw data (ARVI): {str(gdf.iloc[invalid_samples[0][1]]['ARVI'])[:100]}...")

# Convert to NumPy arrays
if not all_features:
    raise ValueError("No valid samples loaded. Check GeoJSON files or GEE export.")
X = np.array(all_features, dtype=np.float32)  # Shape: (num_samples, 5, 5, 144)
y = np.array(all_labels)

print(f"Loaded {len(all_features)} valid samples with shape {X.shape}")

2025-06-26 08:59:54.691945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750928394.876593      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750928394.927823      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Inspecting first 2 rows of first GeoJSON file:

Row 0:
  Band ARVI: type=<class 'numpy.float64'>, raw_value=0.41744154691696167...
  Band ARVI: shape=(5, 5), first few values=[0.41744155 0.41744155 0.41744155 0.41744155 0.41744155]
  Band B1: type=<class 'numpy.float64'>, raw_value=0.12300000339746475...
  Band B1: shape=(5, 5), first few values=[0.123 0.123 0.123 0.123 0.123]
  Band B11: type=<class 'numpy.float64'>, raw_value=0.18479999899864197...
  Band B11: shape=(5, 5), first few values=[0.1848 0.1848 0.1848 0.1848 0.1848]
  Band NDVI: type=<class 'numpy.float64'>, raw_value=0.42132997512817383...
  Band NDVI: shape=(5, 5), first few values=[0.42132998 0.42132998 0.42132998 0.42132998 0.42132998]
  Band B2_1: type=<class 'numpy.float64'>, raw_value=0.11259999871253967...
  Band B2_1: shape=(5, 5), first few values=[0.1126 0.1126 0.1126 0.1126 0.1126]
  Band NDVI_7: type=<class 'numpy.float64'>, raw_value=0.3798370659351349...
  Band NDVI_7: shape=(5, 5), first few values=[0.37983

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import geopandas as gpd
import os
import json
import joblib
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# 2. Handle NaN Values
print("\nChecking for NaN values...")
nan_mask = np.any(np.isnan(X), axis=(1, 2, 3))
nan_count = np.sum(nan_mask)
print(f"Found {nan_count} samples with NaN values")
if nan_count > 0:
    print(f"Removing {nan_count} samples with NaN values")
    valid_mask = ~nan_mask
    X = X[valid_mask]
    y = y[valid_mask]
    print(f"New data shape after removing NaN: {X.shape}")

# 3. Normalize Data
print("\nNormalizing data...")
X_min = np.nanmin(X, axis=(0, 1, 2), keepdims=True)
X_max = np.nanmax(X, axis=(0, 1, 2), keepdims=True)
X = (X - X_min) / (X_max - X_min + 1e-6)  # Min-max normalization to [0, 1]
print(f"Data range after normalization: min={np.nanmin(X):.4f}, max={np.nanmax(X):.4f}")

# 4. Preprocess Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)
num_classes = len(label_encoder.classes_)
print(f"\nData shape: {X.shape}, Number of classes: {num_classes}")

# 5. Define Feature Combinations
spectral_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']  # 12 bands
veg_indices = ['ARVI', 'EVI', 'EVI2', 'NDVI', 'NDWI', 'RENDVI', 'SAVI', 'NBR']  # 8 bands
dem_band = ['DEM']  # 1 band

# Create and verify channel indices
spectral_indices = [i for i, col in enumerate(band_columns) if any(col.startswith(band) for band in spectral_bands)]
veg_indices_indices = [i for i, col in enumerate(band_columns) if any(col.startswith(band) for band in veg_indices)]
dem_indices = [i for i, col in enumerate(band_columns) if col.startswith('DEM')]

# Debug channel indices
print("\nDebugging channel indices:")
print(f"Spectral indices ({len(spectral_indices)}): {[band_columns[i] for i in spectral_indices]}")
print(f"Vegetation indices ({len(veg_indices_indices)}): {[band_columns[i] for i in veg_indices_indices]}")
print(f"DEM indices ({len(dem_indices)}): {[band_columns[i] for i in dem_indices]}")
print(f"Spectral + Veg indices ({len(spectral_indices + veg_indices_indices)}): {[band_columns[i] for i in spectral_indices + veg_indices_indices]}")

# Verify expected channel counts
expected_veg_channels = 8 * len(months)  # 8 bands × 8 months
if len(veg_indices_indices) != expected_veg_channels:
    print(f"Warning: Expected {expected_veg_channels} vegetation indices channels, got {len(veg_indices_indices)}")
    missing_veg_bands = [band for band in veg_indices if not any(col.startswith(band) for col in band_columns)]
    print(f"Missing vegetation indices: {missing_veg_bands}")

feature_combinations = {
    'spectral': (spectral_indices, 96, 'Spectral Bands Only'),
    'spectral_veg': (spectral_indices + veg_indices_indices, 96 + len(veg_indices_indices), 'Spectral + Vegetation Indices'),
    'spectral_dem': (spectral_indices + dem_indices, 96 + 8, 'Spectral + DEM'),
    'all_features': (list(range(144)), 144, 'Spectral + Vegetation Indices + DEM')
}

# 6. Define CNN Model
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# 7. Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

# 8. Train and Evaluate for Each Feature Combination
results = {}
for combo_name, (indices, num_channels, description) in feature_combinations.items():
    print(f"\nTraining model with {description} ({num_channels} channels)")
    
    # Select feature subset
    X_subset = X[:, :, :, indices]
    print(f"Subset shape: {X_subset.shape}")
    
    # Verify no NaN in subset
    if np.any(np.isnan(X_subset)):
        print(f"Warning: NaN values found in {description} subset")
    
    # Train-test-validation split
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y_onehot, test_size=0.15, random_state=42, stratify=y_onehot)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765, random_state=42, stratify=y_train)
    
    # Build and compile model
    model = build_cnn(input_shape=(5, 5, num_channels), num_classes=num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train model
    history = model.fit(
        data_augmentation(X_train), y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ModelCheckpoint(f'best_model_{combo_name}.keras', save_best_only=True)
        ],
        verbose=1
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"{description} Test Accuracy: {test_accuracy:.4f}")
    
    # Confusion matrix
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix: {description}')
    plt.savefig(f'confusion_matrix_{combo_name}.png')
    plt.close()
    
    # Store results
    results[combo_name] = {
        'description': description,
        'test_accuracy': test_accuracy,
        'test_loss': test_loss,
        'history': history.history
    }

# 9. Compare Results
print("\nSummary of Results:")
for combo_name, result in results.items():
    print(f"{result['description']}: Test Accuracy = {result['test_accuracy']:.4f}, Test Loss = {result['test_loss']:.4f}")

# Plot accuracy comparison
plt.figure(figsize=(10, 6))
accuracies = [results[combo]['test_accuracy'] for combo in results]
combo_names = [results[combo]['description'] for combo in results]
plt.bar(combo_names, accuracies)
plt.xlabel('Feature Combination')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy by Feature Combination')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('accuracy_comparison.png')
plt.close()

# 10. Save Label Encoder and Results
joblib.dump(label_encoder, 'label_encoder.pkl')
with open('results_summary.json', 'w') as f:
    json.dump({k: {kk: vv for kk, vv in v.items() if kk != 'history'} for k, v in results.items()}, f, indent=4)


Checking for NaN values...
Found 6440 samples with NaN values
Removing 6440 samples with NaN values
New data shape after removing NaN: (26268, 5, 5, 144)

Normalizing data...
Data range after normalization: min=0.0000, max=1.0000

Data shape: (26268, 5, 5, 144), Number of classes: 19

Debugging channel indices:
Spectral indices (96): ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'B1_1', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B8_1', 'B8A_1', 'B9_1', 'B11_1', 'B12_1', 'B1_2', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'B8_2', 'B8A_2', 'B9_2', 'B11_2', 'B12_2', 'B1_3', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'B8_3', 'B8A_3', 'B9_3', 'B11_3', 'B12_3', 'B1_4', 'B2_4', 'B3_4', 'B4_4', 'B5_4', 'B6_4', 'B7_4', 'B8_4', 'B8A_4', 'B9_4', 'B11_4', 'B12_4', 'B1_5', 'B2_5', 'B3_5', 'B4_5', 'B5_5', 'B6_5', 'B7_5', 'B8_5', 'B8A_5', 'B9_5', 'B11_5', 'B12_5', 'B1_6', 'B2_6', 'B3_6', 'B4_6', 'B5_6', 'B6_6', 'B7_6', 'B8_6', 'B8A_6', 'B9_6', 'B11_6', 'B

I0000 00:00:1750928691.687280      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



Training model with Spectral Bands Only (96 channels)
Subset shape: (26268, 5, 5, 96)
Epoch 1/50


I0000 00:00:1750928699.518656      59 service.cc:148] XLA service 0x784c94012e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750928699.519171      59 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750928699.934266      59 cuda_dnn.cc:529] Loaded cuDNN version 90300


 54/575 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1877 - loss: 2.9053

I0000 00:00:1750928702.663614      59 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


575/575 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.3186 - loss: 2.2618 - val_accuracy: 0.1342 - val_loss: 6.5486
Epoch 2/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4806 - loss: 1.6972 - val_accuracy: 0.3905 - val_loss: 1.9868
Epoch 3/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5280 - loss: 1.5624 - val_accuracy: 0.3367 - val_loss: 2.4204
Epoch 4/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5612 - loss: 1.4455 - val_accuracy: 0.3608 - val_loss: 2.5333
Epoch 5/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5628 - loss: 1.4279 - val_accuracy: 0.4017 - val_loss: 2.0715
Epoch 6/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5910 - loss: 1.3331 - val_accuracy: 0.5633 - val_loss: 1.3933
Epoch 7/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5921 - loss: 1.2959 - val_accuracy: 0.3476 - val_loss: 2.5993
Epoch 8/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6089 - loss: 1.2775 - val_accuracy: 0.4631 - va

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

# 11. Compute Additional Metrics
extended_results = {}
for combo_name, (indices, num_channels, description) in feature_combinations.items():
    print(f"\nComputing additional metrics for {description} ({num_channels} channels)")
    
    # Load the best model
    model = models.load_model(f'best_model_{combo_name}.keras')
    
    # Get test data for this combination
    X_subset = X[:, :, :, indices]
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y_onehot, test_size=0.15, random_state=42, stratify=y_onehot)
    
    # Predict on test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    # Compute precision, recall, F1-score (macro-averaged)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    
    # Compute per-class accuracy from confusion matrix
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
    per_class_accuracy_dict = {label_encoder.classes_[i]: acc for i, acc in enumerate(per_class_accuracy)}
    
    # Store extended results
    extended_results[combo_name] = {
        'description': description,
        'test_accuracy': results[combo_name]['test_accuracy'],
        'test_loss': results[combo_name]['test_loss'],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'per_class_accuracy': per_class_accuracy_dict
    }
    
    # Print metrics
    print(f"{description}:")
    print(f"  Test Accuracy: {extended_results[combo_name]['test_accuracy']:.4f}")
    print(f"  Test Loss: {extended_results[combo_name]['test_loss']:.4f}")
    print(f"  Precision (macro): {precision:.4f}")
    print(f"  Recall (macro): {recall:.4f}")
    print(f"  F1-Score (macro): {f1:.4f}")
    print(f"  Per-Class Accuracy: {per_class_accuracy_dict}")

# 12. Summarize Metrics in a Table
metrics_df = pd.DataFrame({
    'Feature Combination': [extended_results[combo]['description'] for combo in extended_results],
    'Test Accuracy': [extended_results[combo]['test_accuracy'] for combo in extended_results],
    'Test Loss': [extended_results[combo]['test_loss'] for combo in extended_results],
    'Precision': [extended_results[combo]['precision'] for combo in extended_results],
    'Recall': [extended_results[combo]['recall'] for combo in extended_results],
    'F1-Score': [extended_results[combo]['f1_score'] for combo in extended_results]
})
print("\nMetrics Summary Table:")
print(metrics_df.to_string(index=False))

# Save metrics table to CSV
metrics_df.to_csv('metrics_summary.csv', index=False)

# 13. Plot Per-Class Accuracies
plt.figure(figsize=(12, 8))
for combo_name, result in extended_results.items():
    per_class_acc = result['per_class_accuracy']
    plt.plot(list(per_class_acc.keys()), list(per_class_acc.values()), marker='o', label=result['description'])
plt.xlabel('Class')
plt.ylabel('Per-Class Accuracy')
plt.title('Per-Class Accuracy by Feature Combination')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('per_class_accuracy.png')
plt.close()

# 14. Save Extended Results
with open('extended_results_summary.json', 'w') as f:
    json.dump({k: {kk: vv for kk, vv in v.items() if kk != 'history'} for k, v in extended_results.items()}, f, indent=4)


Computing additional metrics for Spectral Bands Only (96 channels)
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Spectral Bands Only:
  Test Accuracy: 0.7176
  Test Loss: 0.8580
  Precision (macro): 0.6467
  Recall (macro): 0.5666
  F1-Score (macro): 0.5748
  Per-Class Accuracy: {'alder': 0.8340425531914893, 'birch': 0.5286885245901639, 'black pine': 0.2553191489361702, 'cherry': 0.043478260869565216, 'douglas fir': 0.8152610441767069, 'english oak': 0.707395498392283, 'european ash': 0.5045045045045045, 'european beech': 0.8125, 'european larch': 0.38461538461538464, 'japanese larch': 0.7151515151515152, 'linden': 0.0, 'norway spruce': 0.759656652360515, 'poplar': 0.13636363636363635, 'red oak': 0.7604790419161677, 'scots pine': 0.9420970266040689, 'sessile oak': 0.8663366336633663, 'silver fir': 0.7466666666666667, 'sycamore maple': 0.3316831683168317, 'weymouth pine': 0.6212121212121212}

Computing additional metrics for Spectral + Vegetation Indices (136 channels)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Spectral + Vegetation Indices:
  Test Accuracy: 0.7285
  Test Loss: 0.8428
  Precision (macro): 0.7088
  Recall (macro): 0.6140
  F1-Score (macro): 0.6136
  Per-Class Accuracy: {'alder': 0.6340425531914894, 'birch': 0.45081967213114754, 'black pine': 0.40425531914893614, 'cherry': 0.17391304347826086, 'douglas fir': 0.4578313253012048, 'english oak': 0.887459807073955, 'european ash': 0.5900900900900901, 'european beech': 0.9419642857142857, 'european larch': 0.7350427350427351, 'japanese larch': 0.7212121212121212, 'linden': 0.10526315789473684, 'norway spruce': 0.8283261802575107, 'poplar': 0.29545454545454547, 'red oak': 0.49700598802395207, 'scots pine': 0.8951486697965572, 'sessile oak': 0.8366336633663366, 'silver fir': 0.8266666666666667, 'sycamore maple': 0.4603960396039604, 'weymouth pine': 0.9242424242424242}

Computing additional metrics for Spectral + DEM (104 channels)
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Spectral + DEM:
  Test 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Spectral + Vegetation Indices + DEM:
  Test Accuracy: 0.7224
  Test Loss: 0.8527
  Precision (macro): 0.6562
  Recall (macro): 0.6060
  F1-Score (macro): 0.6010
  Per-Class Accuracy: {'alder': 0.4808510638297872, 'birch': 0.4385245901639344, 'black pine': 0.5531914893617021, 'cherry': 0.043478260869565216, 'douglas fir': 0.570281124497992, 'english oak': 0.8006430868167203, 'european ash': 0.581081081081081, 'european beech': 0.9308035714285714, 'european larch': 0.6068376068376068, 'japanese larch': 0.8484848484848485, 'linden': 0.0, 'norway spruce': 0.8733905579399142, 'poplar': 0.29545454545454547, 'red oak': 0.6586826347305389, 'scots pine': 0.8622848200312989, 'sessile oak': 0.6336633663366337, 'silver fir': 0.96, 'sycamore maple': 0.5891089108910891, 'weymouth pine': 0.7878787878787878}

Metrics Summary Table:
                Feature Combination  Test Accuracy  Test Loss  Precision   Recall  F1-Score
                Spectral Bands Only    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import joblib
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import json

# 2. Handle NaN Values
print("\nChecking for NaN values...")
nan_mask = np.any(np.isnan(X), axis=(1, 2, 3))
nan_count = np.sum(nan_mask)
print(f"Found {nan_count} samples with NaN values")
if nan_count > 0:
    print(f"Removing {nan_count} samples with NaN values")
    valid_mask = ~nan_mask
    X = X[valid_mask]
    y = y[valid_mask]
    print(f"New data shape after removing NaN: {X.shape}")

# 3. Normalize Data
print("\nNormalizing data...")
X_min = np.nanmin(X, axis=(0, 1, 2), keepdims=True)
X_max = np.nanmax(X, axis=(0, 1, 2), keepdims=True)
X = (X - X_min) / (X_max - X_min + 1e-6)  # Min-max normalization to [0, 1]
print(f"Data range after normalization: min={np.nanmin(X):.4f}, max={np.nanmax(X):.4f}")

# 4. Preprocess Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded)
num_classes = len(label_encoder.classes_)
print(f"\nData shape: {X.shape}, Number of classes: {num_classes}")

# 5. Define Feature Combination (Spectral + Selected Indices + DEM)
bands = ['ARVI', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'DEM', 'EVI', 'EVI2', 'NDVI', 'NDWI']
months = ['', '_1', '_2', '_3', '_4', '_5', '_6', '_7']
band_columns = [band + month for month in months for band in bands]  # 18 bands × 8 months = 144 channels

spectral_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']  # 12 bands
veg_indices = ['NDVI', 'EVI', 'SAVI', 'NDWI']  # 4 important indices
dem_band = ['DEM']  # 1 band

# Create and verify channel indices
spectral_indices = [i for i, col in enumerate(band_columns) if any(col.startswith(band) for band in spectral_bands)]  # 12 × 8 = 96 channels
veg_indices_indices = [i for i, col in enumerate(band_columns) if any(col.startswith(band) for band in veg_indices)]  # 4 × 8 = 32 channels
dem_indices = [i for i, col in enumerate(band_columns) if col.startswith('DEM')]  # 1 × 8 = 8 channels

# Combine for single feature set
feature_indices = spectral_indices + veg_indices_indices + dem_indices
num_channels = 96 + 32 + 8  # 136 channels
description = 'Spectral + Selected Indices + DEM'

# Debug channel indices
print("\nDebugging channel indices:")
print(f"Spectral indices ({len(spectral_indices)}): {[band_columns[i] for i in spectral_indices]}")
print(f"Vegetation indices ({len(veg_indices_indices)}): {[band_columns[i] for i in veg_indices_indices]}")
print(f"DEM indices ({len(dem_indices)}): {[band_columns[i] for i in dem_indices]}")
print(f"Combined indices ({len(feature_indices)}): {[band_columns[i] for i in feature_indices]}")

# Verify expected channel counts
expected_veg_channels = 4 * len(months)  # 4 bands × 8 months
if len(veg_indices_indices) != expected_veg_channels:
    print(f"Warning: Expected {expected_veg_channels} vegetation indices channels, got {len(veg_indices_indices)}")
    missing_veg_bands = [band for band in veg_indices if not any(col.startswith(band) for col in band_columns)]
    print(f"Missing vegetation indices: {missing_veg_bands}")

# 6. Define CNN Model
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# 7. Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

# 8. Train and Evaluate Model
print(f"\nTraining model with {description} ({num_channels} channels)")

# Select feature subset
X_subset = X[:, :, :, feature_indices]
print(f"Subset shape: {X_subset.shape}")

# Verify no NaN in subset
if np.any(np.isnan(X_subset)):
    print(f"Warning: NaN values found in {description} subset")

# Train-test-validation split
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_onehot, test_size=0.15, random_state=42, stratify=y_onehot)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765, random_state=42, stratify=y_train)

# Build and compile model
model = build_cnn(input_shape=(5, 5, num_channels), num_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    data_augmentation(X_train), y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
    ],
    verbose=1
)

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"{description} Test Accuracy: {test_accuracy:.4f}")

# Compute additional metrics
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Precision, recall, F1-score (macro-averaged)
precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')

# Per-class accuracy from confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
per_class_accuracy_dict = {label_encoder.classes_[i]: acc for i, acc in enumerate(per_class_accuracy)}

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix: {description}')
plt.savefig('confusion_matrix.png')
plt.close()

# 9. Summarize Metrics
metrics_dict = {
    'description': description,
    'test_accuracy': test_accuracy,
    'test_loss': test_loss,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'per_class_accuracy': per_class_accuracy_dict
}
print(f"\nMetrics Summary:")
print(f"Description: {description}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"F1-Score (macro): {f1:.4f}")
print(f"Per-Class Accuracy: {per_class_accuracy_dict}")

# Save metrics to CSV
metrics_df = pd.DataFrame([metrics_dict], columns=['description', 'test_accuracy', 'test_loss', 'precision', 'recall', 'f1_score'])
metrics_df.to_csv('metrics_summary.csv', index=False)

# 10. Plot Per-Class Accuracies
plt.figure(figsize=(12, 8))
plt.plot(list(per_class_accuracy_dict.keys()), list(per_class_accuracy_dict.values()), marker='o', label=description)
plt.xlabel('Class')
plt.ylabel('Per-Class Accuracy')
plt.title('Per-Class Accuracy')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('per_class_accuracy.png')
plt.close()

# Save results
with open('results_summary.json', 'w') as f:
    json.dump({k: v for k, v in metrics_dict.items() if k != 'history'}, f, indent=4)

# Save label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')


Checking for NaN values...
Found 0 samples with NaN values

Normalizing data...
Data range after normalization: min=0.0000, max=1.0000

Data shape: (26268, 5, 5, 144), Number of classes: 19

Debugging channel indices:
Spectral indices (96): ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'B1_1', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B8_1', 'B8A_1', 'B9_1', 'B11_1', 'B12_1', 'B1_2', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'B8_2', 'B8A_2', 'B9_2', 'B11_2', 'B12_2', 'B1_3', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'B8_3', 'B8A_3', 'B9_3', 'B11_3', 'B12_3', 'B1_4', 'B2_4', 'B3_4', 'B4_4', 'B5_4', 'B6_4', 'B7_4', 'B8_4', 'B8A_4', 'B9_4', 'B11_4', 'B12_4', 'B1_5', 'B2_5', 'B3_5', 'B4_5', 'B5_5', 'B6_5', 'B7_5', 'B8_5', 'B8A_5', 'B9_5', 'B11_5', 'B12_5', 'B1_6', 'B2_6', 'B3_6', 'B4_6', 'B5_6', 'B6_6', 'B7_6', 'B8_6', 'B8A_6', 'B9_6', 'B11_6', 'B12_6', 'B1_7', 'B2_7', 'B3_7', 'B4_7', 'B5_7', 'B6_7', 'B7_7', 'B8_7', 'B8A_7', 'B9_7', 'B11_7'

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Metrics Summary:
Description: Spectral + Selected Indices + DEM
Test Accuracy: 0.7463
Test Loss: 0.9204
Precision (macro): 0.6970
Recall (macro): 0.6200
F1-Score (macro): 0.6235
Per-Class Accuracy: {'alder': 0.8127659574468085, 'birch': 0.6352459016393442, 'black pine': 0.19148936170212766, 'cherry': 0.0, 'douglas fir': 0.8514056224899599, 'english oak': 0.8585209003215434, 'european ash': 0.3153153153153153, 'european beech': 0.7767857142857143, 'european larch': 0.5641025641025641, 'japanese larch': 0.7696969696969697, 'linden': 0.2631578947368421, 'norway spruce': 0.8605150214592274, 'poplar': 0.5454545454545454, 'red oak': 0.7544910179640718, 'scots pine': 0.8544600938967136, 'sessile oak': 0.9158415841584159, 'silver fir': 0.8933333333333333, 'sycamore maple': 0.599009900990099, 'weymouth pine': 0.3181818181818182}


['label_encoder.pkl']